---
title: Gaussian Elimination Revisited
subject:  Linear Algebraic Systems
subtitle: Gaussian Elimination as Matrix Factorization
short_title: Gaussian Elimination using Matrices
authors:
  - name: Nikolai Matni
    affiliations:
      - Dept. of Electrical and Systems Engineering
      - University of Pennsylvania
    email: nmatni@seas.upenn.edu
license: CC-BY-4.0
keywords: Gaussian Elimination, pivots, algorithm using matrices
math:
  '\vv': '\mathbf{#1}'
  '\bm': '\begin{bmatrix}'
  '\em': '\end{bmatrix}'
  '\R': '\mathbb{R}'
---

## Reading
Material related to this page, as well as additional exercises, can be found in LAA Ch. 2.5, ALA Ch 1.3, and ILA Ch. 2.6.  This page is mostly based on ALA Ch 1.3.

## Learning Objectives

By the end of this page, you should know:
- Gaussian elimination for regular matrices using matrix representation
- what are pivots in a matrix
- algorithm to solve linear equations using Gaussian elimination and back substitution

## Gaussian Elimination: Regular Case
With basic matrix arithmetic operations in our toolkit, we will develop a systematic method for solving linear systems of equations.  For a linear system $A\vv x = \vv b$, with $A$ an $m\times n$ coefficient matrix, $\vv x$ an $n \times 1$ unknowns vector, and $\vv b$ an $m \times 1$ right hand side vector, we define the _augmented matrix_:
```{math}
:label: augmat
M = \left[\begin{array}{c|c} A & \vv b \end{array}\right]
=\left[ \begin{array}{cccc|c} 
a_{11} & a_{12} & \cdots & a_{1n} & b_1 \\
a_{21} & a_{22} & \cdots & a_{2n} & b_2 \\
\vdots & \vdots & \ddots & \vdots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn} & b_m \end{array}\right],
```
which is an $m \times (n+1)$ matrix obtained by tacking the right-hand side vector $\vv b$ onto the right of the coefficient matrix $A$.  The extra vertical line is just to remind us that the last column of this matrix plays a special role.  For example, the augmented matrix for our [example system](./021-linsys-gauss.ipynb#simple-linsys) is
\begin{equation}
\label{augmat-ex}
M = \left[ \begin{array}{ccc|c} 1 & 2 & 1 & 2\\ 2 & 6 & 1 & 7 \\ 1 & 1 & 4 & 3 \end{array}\right]
\end{equation}
Note that it is simple to go back and forth between the original linear system and the augmented matrix, but since operations on equations also affect their right-hand sides, it is convenient to keep track of everything together using the augmented matrix.

For the time being, we will concentrate our efforst on linear systems that have the same number, $n$, of equations as unknowns.  The associated coefficient matrix $A$ is square of size $n \times n$, and the corresponding augmented matrix $M = [ A \, | \, \vv b]$ then has size $n \times (n+1)$.

We start with a simple observation connecting [Linear System Operation \#1](./021-linsys-gauss.ipynb#linop1) to its equivalent matrix operation
```{prf:observation} Elementary Row Operation \#1
:label: rowop1
Adding a scalar multiple of one row of the augmented matrix to another row is the equivalent of adding a multiple of one equation to another in the system of linear equations it defines.  As such, this does not change the solution set and leads to an equivalent augmented matrix.
```

For example, when solving [example system](./021-linsys-gauss.ipynb#simple-linsys), our first step was to subtract two times the first equation from the second.  This is equivalently done by subtracting two times the first row of the augmented matrix [](#augmat-ex) from the second row:
$$
-2\bm 1 & 2 & 1 & 2 \em + \bm 2 & 6 & 1 & 7 \em = \bm 0 & 2 & -1 & 3\em.
$$
We recognize this as the second row of the modified augmented matrix
\begin{equation}
\label{pivot1}
\left[ \begin{array}{ccc|c} 1 & 2 & 1 & 2\\ 0 & 2 & -1 & 3\\ 1 & 1 & 4 & 3 \end{array}\right],
\end{equation}
that corresponds to the [first equivalent example system](./021-linsys-gauss.ipynb#simple-linsys0).  When elementary row operation \#1 is performed, it is critical that the result replaces the row being added to and _not_ the row being multiplied by the scalar.  Notice that the elimination of a variable in an equation, in this case the first variable in the second equation, amounts to making its entry in the coefficient matrix equal to zero.

### Pivots
```{image} ../figures/02-pivot.gif
:alt: Pivot!
:width: 500px
:align: center
```
We will call the $(1,1)$ entry of the coefficient matrix the _first pivot_.  The precise definition of a pivot will become clear as we continue, but one key requirement is that _a pivot must always be nonzero_.  Eliminating the first variable $x_1$ from the second and third equations is the same as making all of the matrix entries in the column below the pivot equal to zero.  We have already done this with the $(2,1)$ entry in [](#pivot1).  To make the $(3,1)$ entry equal to zero, we subtract the first from from the last row, resulting in the augmented matrix
\begin{equation}
\label{pivot2}
\left[ \begin{array}{ccc|c} 1 & 2 & 1 & 2\\ 0 & 2 & -1 & 3\\ 0 & -1 & 3 & 1 \end{array}\right],
\end{equation}
which we again recognize as the corresponding to the [second equivalent example system](./021-linsys-gauss.ipynb#simple-linsys1).  The _second pivot_ is the $(2,2)$ entry of this matrix, which is $2$, and is the coefficient of the second variable $x_2$ in the second equation.  Again, the pivot must be nonzero.  We use the [](#rowop1) of adding $1/2$ of the second row to the third row to make the entry below the second pivot equal to 0, resulting in the augmented matrix
\begin{equation}
\label{pivot3}
\left[ \begin{array}{ccc|c} 1 & 2 & 1 & 2\\ 0 & 2 & -1 & 3\\ 0 & 0 & \frac{5}{2} & \frac{5}{2} \end{array}\right],
\end{equation}
that corresponds to the [triangular system equivalent to our example system](./021-linsys-gauss.ipynb#simple-linsys2).  We write the final augmented matrix as
$$
N = [U \, | \, \vv c], \quad \text{where} \quad U = \bm 1 & 2 & 1 \\ 0 & 2 & -1 \\ 0 & 0 & \frac{5}{2}\em, \quad \vv c = \bm 2 \\ 3 \\ \frac{5}{2} \em.
$$

The corresponding linear system can be written as $U\vv x = \vv c$.  A special feature of this system is that the coefficient matrix $U$ is _upper triangular_[^upper], which means that all entries below the main diagonal are zero, i.e., $u_{ij}=0$ whenever $i>j$.  The three nonzero entries on its diagonal, $1$, $2$, and $5/2$, including the last one in the $(3,3)$ slot, are the three pivots.  Once the system has been reduced to this triangular form, we can easily solve it via Back Substitution.

[^upper]: It's convention we used the symbol $U$ to remind ourselves that the matrix is upper triangular.

What we just described is an algorithm for solving a linear system of $n$ equations in $n$ unknowns, and is known as _regular Gaussian Elimination_.  We'll call a square matrix $A$ _regular_ if the algorithm successfully reduces it to the upper triangular form $U$ with all nonzero pivots on the diagonal.  If this fails to happen, i.e., if a pivot appearing on the diagonal is zero, then the matrix is not regular.  We then use the pivot row to make all entries lying in the column below the pivot equal to zero through elementary row operations.  The solution is then found by applying Back Substitution to the resulting system.  We'll summarize both of these algorithms in _pseudocode_ below.  Later, we'll see how to translate this pseudocode into actual Python code that can be run on a computer.

:::{prf:algorithm} Regular Gaussian Elimination
:label: reg-ge

**Inputs** Augmented matrix $M = [ A \, | \, \vv b]$

**Output** Equivalent upper triangular form $M = [U \, | \, \vv c]$ if $A$ is regular, "$A$ is not regular" token otherwise

for $j=1$ to $n$:\
$\quad$ if $m_{jj}=0$:\
$\quad \quad$ **return** "$A$ is not regular"\
$\quad$ else for $i= j + 1$ to $n$:\
$\quad \quad$ set $l_{ij}\leftarrow m_{ij}/m_{jj}$\
$\quad \quad$ add $-l_{ij}$ times row $j$ of $M$ to row $i$ of $M$\
**return** $M = [U \, | \, \vv c]$ 
:::
Here we use what are called _in place updates_, meaning that the same letter $M$ (with entries $m_{ij}$) denotes the current augmented matrix at each stage in the computation.  We initialize with $M=[A \, | \, \vv b]$, and output (assuming $A$ is regular) the upper triangular equivalent augmented matrix $M = [U \, | \, \vv c]$, where $U$ is the upper triangular matrix with diagonal entries the pivots, and $\vv c$ is the resulting vector of the right-hand sides of the triangular system $U\vv x = \vv c$.

Next, let's take a look at the pseudocode for Back Substitution.
:::{prf:algorithm} Back Substitution
:label: back-sub

**Inputs** Triangular form augmented matrix $M = [U \, | \, \vv c]$.  $U$ is assumed to have nonzero diagonals $u_{ii}\neq 0$.

**Output** Solution $\vv x$ to $U\vv x = \vv c$.

set $x_n\leftarrow c_n/u_{nn}$\
for $i=n-1$ to $1$: (decrementing by $-1$ at each iteration)\
$\quad$ set $x_i \leftarrow \frac{1}{u_{ii}}\left(c_i-\displaystyle\sum_{j=i+1}^{n}u_{ij}x_j\right)$\
**return** solution $\vv x$

### Worked Examples

````{exercise}  TODO
:label: row-reduce-ex1
Write me
:::{hint} Click me for a hint!
:class: dropdown
Write me

:::
```{solution} my-exercise
:class: dropdown
Write me
```
````

````{exercise}  TODO
:label: row-reduce-ex1
Write me
:::{hint} Click me for a hint!
:class: dropdown
Write me

:::
```{solution} my-exercise
:class: dropdown
Write me
```
````

In [44]:
# Gaussian elimination and back substitution

import numpy as np

def GaussElem(A, b):
    M = np.hstack((A, b))
    
    n = A.shape[0] # number of rows of A = number of equations
    
    # Gaussian elimination
    for j in range(n):
        if M[j, j] == 0:
            print("A is not regular")
            break
        else:
            for i in range(j+1, n):
                scalar = M[i, j]/M[j, j]
                M[i, :] = M[i, :] - scalar*M[j, :]

    # back substitution
    x = np.zeros((n,))

    x[n-1] = M[n-1, -1]/M[n-1, n-1] # -1 is the last index of the particular axis in a python array
    for i in range(n-2, -1, -1): # range(a, b, c) returns the list [a, a+c ,a+2c, ..., b) 
        x[i] = (M[i, -1] - np.sum(M[i, i+1:-1]*x[i+1:]))/M[i, i] # * is elementwise multiplication of arrays
    
    return x

In [45]:
A = np.array([[1, 2, 1],
              [2, 6, 1],
              [1, 1, 4]])
b = np.array([[2],
              [7],
              [3]])

x = GaussElem(A, b)
x

array([-3.,  2.,  1.])

In [46]:
# solve linear system of equations Ax=b using NumPy's inbuilt function

x_numpy = np.linalg.solve(A, b) 
x_numpy

array([[-3.],
       [ 2.],
       [ 1.]])